# 💫  Explore and analyze `spaCy` NER pipelines

In this tutorial, you'll learn to log [spaCy](https://spacy.io/) Name Entity Recognition (NER) predictions. 

This is useful for: 

- Evaluating pre-trained models.
- Spotting frequent errors both during development and production. 
- Improve your pipelines over time using Rubrix annotation mode.
- Monitor your model predictions using Rubrix integration with Kibana

Let's get started!


![spacy_ner explore](https://github.com/recognai/rubrix-materials/raw/main/tutorials/1/spacy1.gif)

## Introduction

In this tutorial we will:

- Load the [*Gutenberg Time*](https://huggingface.co/datasets/gutenberg_time) dataset from the Hugging Face Hub.

- Use a transformer-based spaCy model for detecting entities in this dataset and log the detected entities into a Rubrix dataset. This dataset can be used for exploring the quality of predictions and for creating a new training set, by correcting, adding and validating entities.

- Use a smaller spaCy model for detecting entities and log the detected entities into the same Rubrix dataset for comparing its predictions with the previous model.

- As a bonus, we will use Rubrix and spaCy on a more challenging dataset: IMDB.

## Setup Rubrix

**If you are new to Rubrix, visit and ⭐ star Rubrix for more materials like and detailed docs**: [Github repo](https://github.com/recognai/rubrix)

If you have not installed and launched Rubrix, check the [Setup and Installation guide](https://docs.rubrix.ml/en/latest/getting_started/setup%26installation.html).


Once installed, you only need to import Rubrix:

In [ ]:
import rubrix as rb

## Install tutorial dependencies

In this tutorial, we'll use the `datasets` and `spaCy` libraries and the `en_core_web_trf` pretrained English model, a Roberta-based spaCy model . If you do not have them installed, run:

In [ ]:
%pip install datasets -qqq
%pip install -U spacy -qqq
%pip install protobuf

## Our dataset

For this tutorial, we're going to use the [*Gutenberg Time*](https://huggingface.co/datasets/gutenberg_time) dataset from the Hugging Face Hub. It contains all explicit time references in a dataset of 52,183 novels whose full text is available via Project Gutenberg. From extracts of novels, we are surely going to find some NER entities.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("gutenberg_time", split="train")

Let's take a look at our dataset! 

In [ ]:
train, test = dataset.train_test_split(test_size=0.002, seed=42).values() ; test

## Logging spaCy NER entities into Rubrix

### Using a Transformer-based pipeline

Let's install and load our roberta-based pretrained pipeline and apply it to one of our dataset records:

In [ ]:
!python -m spacy download en_core_web_trf

In [ ]:
import spacy

nlp = spacy.load("en_core_web_trf")
doc = nlp(dataset[0]["tok_context"])
doc

Now let's apply the nlp pipeline to our dataset records, collecting the tokens and NER entities. 

In [ ]:
records = []
for record in test:
    # We only need the text of each instance
    text = record["tok_context"]
    
    # spaCy Doc creation
    doc = nlp(text)    
    
    # Entity annotations
    entities = [
        (ent.label_, ent.start_char, ent.end_char)  
        for ent in doc.ents
    ] 

    # Pre-tokenized input text
    tokens = [token.text  for token in doc]
    
    # Rubrix TokenClassificationRecord list
    records.append(
        rb.TokenClassificationRecord(
            text=text,
            tokens=tokens,
            prediction=entities,
            prediction_agent="en_core_web_trf",
        )
    )

In [ ]:
records[0]

In [ ]:
rb.log(records=records, name="gutenberg_spacy_ner")

If you go to the `gutenberg_spacy_ner` dataset in Rubrix you can explore the predictions of this model:

- You can filter records containing specific entity types.
- You can see the most frequent "mentions" or surface forms for each entity. Mentions are the string values of specific entity types, such as for example "1 month" can be the mention of a duration entity. This is useful for error analysis, to quickly see potential issues and problematic entity types.
- You can use the free-text search to find records containing specific words.
- You could validate, include or reject specific entity annotations to build a new training set.

### Using a smaller but more efficient pipeline

Now let's compare with a smaller, but more efficient pre-trained model. Let's first download it

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(dataset[0]["tok_context"])

In [ ]:
records = []    # Creating and empty record list to save all the records

for record in test:

    text = record["tok_context"]  # We only need the text of each instance
    doc = nlp(text)    # spaCy Doc creation
    
    # Entity annotations
    entities = [
        (ent.label_, ent.start_char, ent.end_char)  
        for ent in doc.ents
    ] 

    # Pre-tokenized input text
    tokens = [token.text  for token in doc]
    

    # Rubrix TokenClassificationRecord list
    records.append(
        rb.TokenClassificationRecord(
            text=text,
            tokens=tokens,
            prediction=entities,
            prediction_agent="en_core_web_sm",
        )
    )

In [ ]:
rb.log(records=records, name="gutenberg_spacy_ner")

## Exploring and comparing `en_core_web_sm` and `en_core_web_trf` models

If you go to your `gutenberg_spacy_ner` you can explore and compare the results of both models. 

You can use the `predicted by` filter, which comes from the `prediction_agent` parameter of your `TextClassificationRecord` to only see predictions of a specific model:


![spacy_models_meta](img/spacy_ner2.png "spaCy models predicted_by filter")


## Extra: Explore the IMDB dataset

So far both spaCy pretrained models seem to work pretty well. Let's try with a more challenging dataset, which is more dissimilar to the original training data these models have been trained on. 

In [ ]:
imdb = load_dataset("imdb", split="test[0:5000]")

In [ ]:
records = []
for record in imdb:
    # We only need the text of each instance
    text = record["text"]
    
    # spaCy Doc creation
    doc = nlp(text)    
    
    # Entity annotations
    entities = [
        (ent.label_, ent.start_char, ent.end_char)  
        for ent in doc.ents
    ] 

    # Pre-tokenized input text
    tokens = [token.text  for token in doc]
    
    # Rubrix TokenClassificationRecord list
    records.append(
        rb.TokenClassificationRecord(
            text=text,
            tokens=tokens,
            prediction=entities,
            prediction_agent="en_core_web_sm",
        )
    )

In [ ]:
rb.log(records=records, name="imdb_spacy_ner")

Exploring this dataset highlights the need of fine-tuning for specific domains.

For example, if we check the most frequent mentions for Person, we find two highly frequent missclassified entities: gore (the film genre) and Oscar (the prize). You can check yourself each an every example by using the filters and search-box.


![spacy imdb](https://github.com/recognai/rubrix-materials/raw/main/tutorials/1/spacy1.gif)

## Summary
In this tutorial, we have learnt to log and explore differnt `spaCy` NER models with Rubrix. Using what we´ve learnt here you can:

- Build custom dashboards using Kibana to monitor and visualize spaCy models.
- Build training sets using pre-trained spaCy models.

## Next steps

### 📚 [Rubrix documentation](https://docs.rubrix.ml) for more guides and tutorials.

### 🙋‍♀️ Join the Rubrix community! A good place to start is the [discussion forum](https://github.com/recognai/rubrix/discussions).

### ⭐ Rubrix [Github repo](https://github.com/recognai/rubrix) to stay updated.